# Transcript from Lecture, February 21, 2023

In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2023-winter/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import json
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

np.set_printoptions(precision = 4)

# Floating-point arithmetic is a leaky abstraction of the real numbers

In [2]:
a = 4/3
a

1.3333333333333333

In [3]:
b = a-1
b

0.33333333333333326

In [4]:
c = 3*b
c

0.9999999999999998

In [5]:
d = 1-c
d

2.220446049250313e-16

# 64-bit 2s complement integers

<b> Base 16 is "hexadecimal", with digits 0,1,2,3,...,f, four bits per digit

In [6]:
cs111.bits

{'0': '0000',
 '1': '0001',
 '2': '0010',
 '3': '0011',
 '4': '0100',
 '5': '0101',
 '6': '0110',
 '7': '0111',
 '8': '1000',
 '9': '1001',
 'a': '1010',
 'b': '1011',
 'c': '1100',
 'd': '1101',
 'e': '1110',
 'f': '1111'}

<b> int64 represents integers from $-2^{63}$ to $2^{63}-1$ using 64 bits = 16 hexadecimal digits

In [7]:
print(cs111.int64_to_hex(0))

0000000000000000


In [8]:
print(cs111.int64_to_hex(3))

0000000000000003


In [9]:
print(cs111.int64_to_hex(10))

000000000000000a


In [10]:
print(cs111.int64_to_hex(1024))

0000000000000400


In [11]:
print(cs111.int64_to_hex(-1))

ffffffffffffffff


In [12]:
print(cs111.int64_to_hex(-2))

fffffffffffffffe


In [13]:
# largest positive int64
print(cs111.int64_to_hex(2**63 - 1))

7fffffffffffffff


In [14]:
# most negative int64
print(cs111.int64_to_hex(-2**63))

8000000000000000


# IEEE Standard 64-bit floating-point

In [15]:
cs111.print_float64(0)


input     : 0
as float64: 0.0000000000000000e+00
as hex    : 0000000000000000
sign      : 0 means +
exponent  : 000 means zero or denormal



In [16]:
cs111.print_float64(1)

input     : 1
as float64: 1.0000000000000000e+00
as hex    : 3ff0000000000000
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000000



In [17]:
cs111.print_float64(42)

input     : 42
as float64: 4.2000000000000000e+01
as hex    : 4045000000000000
sign      : 0 means +
exponent  : 404 means 1028 - 1023 = 5
mantissa  : 1.0101000000000000000000000000000000000000000000000000



In [18]:
cs111.print_float64(-1/3)

input     : -0.3333333333333333
as float64: -3.3333333333333331e-01
as hex    : bfd5555555555555
sign      : 1 means -
exponent  : 3fd means 1021 - 1023 = -2
mantissa  : 1.0101010101010101010101010101010101010101010101010101



<b> float64 has both +0 and -0, but they compare as equal

In [19]:
cs111.print_float64(0.0)
cs111.print_float64(-0.0)

input     : 0.0
as float64: 0.0000000000000000e+00
as hex    : 0000000000000000
sign      : 0 means +
exponent  : 000 means zero or denormal

input     : -0.0
as float64: -0.0000000000000000e+00
as hex    : 8000000000000000
sign      : 1 means -
exponent  : 000 means zero or denormal



In [20]:
-0.0 == 0.0

True

# Floating-point infinity and NaN (not-a-number)

In [21]:
np.inf

inf

In [22]:
1/np.inf

0.0

In [23]:
-np.inf == np.inf

False

In [24]:
1/ (-np.inf)

-0.0

In [25]:
# This is a bad flaw in python; it should give inf!

1.0 / 0.0

ZeroDivisionError: float division by zero

In [26]:
np.inf + 100

inf

In [27]:
-2 * np.inf

-inf

In [28]:
0 * np.inf

nan

In [29]:
cs111.print_float64(np.inf)

input     : inf
as float64: inf
as hex    : 7ff0000000000000
sign      : 0 means +
exponent  : 7ff means inf or nan



In [30]:
cs111.print_float64(-np.inf)

input     : -inf
as float64: -inf
as hex    : fff0000000000000
sign      : 1 means -
exponent  : 7ff means inf or nan



In [31]:
cs111.print_float64(np.nan)

input     : nan
as float64: nan
as hex    : 7ff8000000000000
sign      : 0 means +
exponent  : 7ff means inf or nan



<b> Is infinity equal to infinity?

In [32]:
np.inf == np.inf

True

<b> NaN is not equal to anything, including itself!

In [33]:
np.nan == 0

False

In [34]:
np.nan == np.nan

False

In [35]:
np.isnan(np.nan)

True

In [36]:
np.isnan(np.inf)

False

In [37]:
np.isnan(3.14)

False

# Properties of floating-point arithmetic

<b> Numbers that get big

In [ ]:
x = 1.0
while x < 2*x:
    print('x:', x, '    2x:', 2*x)
    lastx = x
    x = 2*x
print('x:', x, '    2x:', 2*x)

In [39]:
cs111.print_float64(lastx)

input     : 8.98846567431158e+307
as float64: 8.9884656743115795e+307
as hex    : 7fe0000000000000
sign      : 0 means +
exponent  : 7fe means 2046 - 1023 = 1023
mantissa  : 1.0000000000000000000000000000000000000000000000000000



<b> Numbers that get little

In [ ]:
x = 1.0
while x > x/2:
    print('x:', x, '    x/2:', 2*x)
    lastx = x
    x = x/2
print('x:', x, '    x/2:', x/2)

In [41]:
cs111.print_float64(lastx)

input     : 5e-324
as float64: 4.9406564584124654e-324
as hex    : 0000000000000001
sign      : 0 means +
exponent  : 000 means zero or denormal



<b>Machine epsilon

In [42]:
x = 1.0
while 1 + x > 1:
    print('x:', x, '    1 + x:', 1+x)
    x = x/2
print('x:', x, '    1 + x:', 1+x)

x: 1.0     1 + x: 2.0
x: 0.5     1 + x: 1.5
x: 0.25     1 + x: 1.25
x: 0.125     1 + x: 1.125
x: 0.0625     1 + x: 1.0625
x: 0.03125     1 + x: 1.03125
x: 0.015625     1 + x: 1.015625
x: 0.0078125     1 + x: 1.0078125
x: 0.00390625     1 + x: 1.00390625
x: 0.001953125     1 + x: 1.001953125
x: 0.0009765625     1 + x: 1.0009765625
x: 0.00048828125     1 + x: 1.00048828125
x: 0.000244140625     1 + x: 1.000244140625
x: 0.0001220703125     1 + x: 1.0001220703125
x: 6.103515625e-05     1 + x: 1.00006103515625
x: 3.0517578125e-05     1 + x: 1.000030517578125
x: 1.52587890625e-05     1 + x: 1.0000152587890625
x: 7.62939453125e-06     1 + x: 1.0000076293945312
x: 3.814697265625e-06     1 + x: 1.0000038146972656
x: 1.9073486328125e-06     1 + x: 1.0000019073486328
x: 9.5367431640625e-07     1 + x: 1.0000009536743164
x: 4.76837158203125e-07     1 + x: 1.0000004768371582
x: 2.384185791015625e-07     1 + x: 1.000000238418579
x: 1.1920928955078125e-07     1 + x: 1.0000001192092896
x: 5.96046447753

<b> *machine epsilon* is the smallest x such that 1 + x > 1

In [43]:
eps = 2**(-52)

print('eps:')
cs111.print_float64(eps)

print('1 + eps:')
cs111.print_float64(1 + eps)

1 + eps == 1

eps:
input     : 2.220446049250313e-16
as float64: 2.2204460492503131e-16
as hex    : 3cb0000000000000
sign      : 0 means +
exponent  : 3cb means 971 - 1023 = -52
mantissa  : 1.0000000000000000000000000000000000000000000000000000

1 + eps:
input     : 1.0000000000000002
as float64: 1.0000000000000002e+00
as hex    : 3ff0000000000001
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000001



False

In [44]:
half_eps = 2**(-53)

print('half_eps:')
cs111.print_float64(half_eps)

print('1 + half_eps:')
cs111.print_float64(1 + half_eps)

1 + half_eps == 1

half_eps:
input     : 1.1102230246251565e-16
as float64: 1.1102230246251565e-16
as hex    : 3ca0000000000000
sign      : 0 means +
exponent  : 3ca means 970 - 1023 = -53
mantissa  : 1.0000000000000000000000000000000000000000000000000000

1 + half_eps:
input     : 1.0
as float64: 1.0000000000000000e+00
as hex    : 3ff0000000000000
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000000



True